## Coursera Capstone Project
By, Anjali 

#### This notebook contains code for Module 3 Assignment of Capstone project

### Libraries

In [1]:
import numpy as np
import pandas as pd
# add support up to 30 columns
pd.set_option("display.max_columns",30)

### Build DataFrame - Q1

Required to explore and cluster the neighborhoods in Toronto

In [2]:
#!pip install bs4
#!pip installl lxml
#!pip install html5lib
#!pip install requests

In [4]:
from bs4 import BeautifulSoup
import requests
# link to the resource
doc_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# get the resource
source = requests.get(doc_url).text
# parse it:
soup = BeautifulSoup(source, 'lxml')

In [9]:
table = soup.find('tbody')
table_rows = table.find_all('tr')
Length = len(table_rows)
print("Length:", Length)

Length: 288


In [12]:
raw_df = []
for tr in table_rows:
    row = tr.text.split('\n')[1:-1] # first and last lines are empty string
    raw_df.append(row)
raw_df = pd.DataFrame(raw_df[1:], columns=raw_df[0])
print("Raw Data")
raw_df.head(n=12)

Raw Data


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


#### Data Cleaning

##### For the purpose of the project, we have to:

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two       neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above   table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page,   the value of the Borough and the Neighborhood columns will be Queen's Park.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [13]:
raw_df = raw_df[raw_df.Borough != 'Not assigned']
raw_df.reset_index(drop=True, inplace=True)
raw_df.head(n=12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [14]:
# group by Postcode
grouped = raw_df.groupby(['Postcode'])


# combine the neighborhoods grouped by postcode and into a new df
neighborhood_grouped = grouped['Neighbourhood'].apply(lambda x: x.sum()) 

neighborhood_grouped = grouped['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

# matches a borough to each postcode
borough_grouped = grouped['Borough'].apply(lambda x: set(x).pop())

# turn borough_grouped and neighborhood_grouped into dataframes
borough = borough_grouped.to_frame()
neighborhood = neighborhood_grouped.to_frame()

grouped_final = borough.merge(neighborhood, on="Postcode")
grouped_final.head(n=12)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [15]:
grouped_final.shape

(103, 2)

### Geo Data - Q2

In [23]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [24]:
geospatial_data = geospatial_data.rename(columns={geospatial_data.columns[0]: "Postcode" })

In [26]:
full_table = grouped_final.merge(geospatial_data, on = 'Postcode')

full_table.head(n=12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Clustering Data - Q3

In [30]:
#!pip install geopy 
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import json 
import requests 
from pandas.io.json import json_normalize 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [31]:
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto is {}, {}.'.format(latitude, longitude))

Toronto is 43.653963, -79.387207.


In [32]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add neighborhood markers to the Toronto map
for lat, long, bor, neigh in zip(full_table['Latitude'], full_table['Longitude'], 
                                 full_table['Borough'], full_table['Neighbourhood']):
    label = '{}, {}'.format(neigh, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 7, 
        popup = label,
        color = 'red',
        fill = True,
        fill_color = 'white',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
        
map_toronto